In [ ]:
%pip install metpy
%pip install sounderpy


In [ ]:
import sounderpy as spy
import metpy.calc as mpcalc
from metpy.units import units
import json
from urllib.request import urlopen
import datetime
from datetime import date, datetime, timezone, timedelta
import requests
from bs4 import BeautifulSoup
data = json.loads((urlopen('https://aviationweather.gov/data/cache/stations.cache.json')).read())
icao = [i['icaoId'] for i in data]
models = {'HRRR':'(F00-48)', 'RAP':'(F00-51)', 'NAM':'(F00-48)', 'NAMNEST':'(F00-60)', 'GFS':'(F00-180)', 'SREF':'(F00-84)', 'HIRESW':'(F00-48)'}
roabsites = ['UIL', 'OTX', 'SLE', 'MFR', 'OAK', 'NKX', 'BOI', 'REV', 'LKN', 'VEF', 'TFX', 'GGW', 'RIW', 'SLC', 'DNR', 'GJT', 'FGZ', 'TUS', 'ABQ', 
             'EPZ', 'BIS', 'ABR', 'UNR', 'LBF', 'OAX', 'DDC', 'TOP', 'OUN', 'AMA', 'MAF', 'FWD', 'DRT', 'CRP', 'BRO', 'INL', 'MPX', 'DVN', 'SGF', 
             'LZK', 'SHV', 'LCH', 'LIX', 'GRB', 'ILX', 'OHX', 'JAN', 'BMX', 'APX', 'DTX', 'ILN', 'CAR', 'GYX', 'CHH', 'ALY', 'BUF', 'OKX', 'PBZ', 
             'ILN', 'LWX', 'WAL', 'RNK', 'GSO', 'MHX', 'FFC', 'CHS', 'TAE', 'JAX', 'TBW', 'MFL', 'EYW', 'HLI', 'HTO', 'GUM', 'STU', 'SJU', 'ACD', 
             'ABE', 'AKN', 'ADQ', 'AFC', 'AMC', 'AFG', 'AYA', 'AOM', 'AOT', 'ASN', 'ABR', 'ANT']

In [ ]:
utchr = (datetime.now(timezone.utc)).strftime('%H')
utcday = (datetime.now(timezone.utc)).strftime('%Y%m%d')

print(utchr, utcday)
utchr = '04'
if int(utchr) in range(1,7):
    current = str(utcday)+"00"
elif int(utchr) in range(7,13):
    current = str(utcday)+"00"
else:
    current = str(utcday)+"12"

print(current[:4],current[4:6],current[6:8],current[8:])

#if error
tdelt = datetime.strptime(current,"%Y%m%d%H")
past = (tdelt - timedelta(hours=12)).strftime('%Y%m%d%H')
print(past[:4],past[4:6],past[6:8],past[8:])

#datetime_object = datetime.strptime(date_string, format_string)
#date_string = "2025-05-16 12:30:45"
#format_string = "%Y-%m-%d %H:%M:%S"


In [ ]:
#################   Available Models   ##################
#     hrrr (F00-48)
#     rap (F00-51)
#     nam (F00-48)
#     namnest (F00-60)
#     gfs (F00-180)
#     sref (F00-84)
#     hiresw (F00-48)

#################     Get Data     #######################
#      ROABs ------   spy.get_obs_data(station, year, month, day, hour, hush=False, clean_it=True)
#      Models -----   spy.get_bufkit_data('model', 'station', forecast_hr, 'run_year', 'run_month', 'run_day', 'run_hour',hush=False, clean_it=True)
#      Reanalysis -   spy.get_model_data('model', latlon, year, month, day, hour, dataset=None, box_avg_size=0.10, hush=False, clean_it=True)

#      hush (bool, optional, default is False) – whether to ‘hush’ a read-out of thermodynamic and kinematic parameters when getting data.
#      clean_it (bool, optional, default is True) – whether to return the raw_data object or a clean_data dict.

In [ ]:
dattype = [{1:'ROAB Soundings',2:'ACARS Partial Soundings',3:'Model Based Soundings'}]



while True:
    try:
        for i in dattype:
            for x in i:
                print(str(x)+':',i[x])
        datkind = int(input('What kind of data do you want? <1, 2, or 3>'))
        if datkind == 1 or datkind == 2 or datkind == 3:
            break
            #continue
        else:
            print('Please only type 1, 2, or 3')
    except ValueError:
        print("Invalid input. Please only type 1, 2, or 3")

print('NEXT!!!!!')


In [ ]:
url = 'https://www.nco.ncep.noaa.gov/omb/dataqc/stations/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
f = soup.get_text()
d = (f[f.find('LAST_USE is date of last citation in Raob Recap within 365 days.')+64:f.find('Page last modified: ')]).split('\n')
d2 = [i.split() for i in list(filter(None, [(" ".join(i.split())).replace(",", "").replace("/", " ") for i in d]))]
keys = ['WMO#','ICAO','IATA','Site','State','RGN','Last_Use']
region = ['E = Eastern', 'S = Southern', 'C = Central', 'W = Western', 'A = Alaska','P = Pacific', 'L = Military', 'F = Profiler', 'M = Mexico', 'N = Canada','B = Caribbean', 'G = Greenland/Iceland']

d5=[]
for i in d2:
    if i[0]=='WMO#':
        continue
    else:
        if len(i)>7:
            t = [' '.join(i[3:len(i)-3])]
            i = i[:3] + t + i[len(i)-3:]
        d5.append(i)
d6 = {}
for i in d5:
    if d5.index(i)==len(d5)-1:
        break
    else:
        d6[i[2]] = i
for i in d6:
    d6[i]= dict(zip(keys,d6[i]))


y = int(input('Do you want to search by: 1 - Site, 2 - State, or 3 - Region <1, 2, 3>'))

if y == 1:
    b = input('What site are you looking for?').capitalize()
elif y == 2:
    b = input('What state are you looking for? <State Abbreviations Only>').upper()
elif y == 3:
    print('Regions are:')
    for i in region:
        print(i)
    b = input('What region are you looking for? <Region Abbreviations Only>').upper()


for i in d6:
    if b == d6[i]['State']:
        print(str(d6[i]['Site'])+','+str(d6[i]['State'])+' -- '+str(d6[i]['IATA'])+':',d6[i]['WMO#'])
    elif b == d6[i]['Site']:
        print(str(d6[i]['Site'])+','+str(d6[i]['State'])+' -- '+str(d6[i]['IATA'])+':',d6[i]['WMO#'])
    elif b == d6[i]['RGN']:
        print(str(d6[i]['Site'])+','+str(d6[i]['State'])+' -- '+str(d6[i]['IATA'])+':',d6[i]['WMO#'])


In [ ]:
def roabsites():
    url = 'https://www.nco.ncep.noaa.gov/omb/dataqc/stations/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    f = soup.get_text()
    d = (f[f.find('LAST_USE is date of last citation in Raob Recap within 365 days.')+64:f.find('Page last modified: ')]).split('\n')
    d2 = [i.split() for i in list(filter(None, [(" ".join(i.split())).replace(",", "").replace("/", " ") for i in d]))]
    keys = ['WMO#','ICAO','IATA','Site','State','RGN','Last_Use']
    region = ['E = Eastern', 'S = Southern', 'C = Central', 'W = Western', 'A = Alaska','P = Pacific', 'L = Military', 'F = Profiler', 'M = Mexico', 'N = Canada','B = Caribbean', 'G = Greenland/Iceland']
    
    d5=[]
    for i in d2:
        if i[0]=='WMO#':
            continue
        else:
            if len(i)>7:
                t = [' '.join(i[3:len(i)-3])]
                i = i[:3] + t + i[len(i)-3:]
            d5.append(i)
    d6 = {}
    for i in d5:
        if d5.index(i)==len(d5)-1:
            break
        else:
            d6[i[2]] = i
    for i in d6:
        d6[i]= dict(zip(keys,d6[i]))

    y = int(input('Do you want to search by:\n1 - Site\n2 - State\n3 - Region\n<1, 2, 3>'))
    
    if y == 1:
        b = input('What site are you looking for?').capitalize()
    elif y == 2:
        b = input('What state are you looking for? <State Abbreviations Only>').upper()
    elif y == 3:
        print('Regions are:')
        for i in region:
            print(i)
        b = input('What region are you looking for? <Region Abbreviations Only>').upper()
    
    
    for i in d6:
        if b == d6[i]['State']:
            print(str(d6[i]['Site'])+','+str(d6[i]['State'])+' -- '+str(d6[i]['IATA'])+':',d6[i]['WMO#'])
        elif b == d6[i]['Site']:
            print(str(d6[i]['Site'])+','+str(d6[i]['State'])+' -- '+str(d6[i]['IATA'])+':',d6[i]['WMO#'])
        elif b == d6[i]['RGN']:
            print(str(d6[i]['Site'])+','+str(d6[i]['State'])+' -- '+str(d6[i]['IATA'])+':',d6[i]['WMO#'])
        sndsite = d6[i]['WMO#']
    return sndsite
    

In [ ]:
#spy.get_obs_data(station, year, month, day, hour, hush=False, clean_it=True)

def ROAB():
    print("Please enter a WFO site")
    while True:
        try:
            wfo = input().upper()
            if wfo in roabsites:
                print('Its in there!')
                break
            elif wfo == 'LIST':
                roabsites()
            else:
                print('Please only enter a valid WFO')
        except ValueError:
            print("Invalid input. Please only enter a valid WFO")
    cur = input("Do you want the latest sounding? <Y or N>").upper()
    if cur == 'Y':
        

def ACARS():
    print("Hello from a function")

def Model():
    print('Here is a list of available models')
    print(list(models.keys()))
    #for i in models:
        #print(i)
    while True:
        try:
            mdl = (input('What model do you want to use?')).upper()
            if mdl in models:
                print('Its in there!')
                break
            else:
                print('Please only enter a model name from the list')
        except ValueError:
            print("Invalid input. Please only enter a model name from the list")
    while True:
        try:
            loc = (input('Where do you want the data from? [enter a valid airport ICAO code]')).upper()
            if loc in icao:
                print('Its in there!')
                break
            else:
                print('Please enter a valid airport ICAO code - ie KCID')
        except ValueError:
            print('Please enter a valid airport ICAO code - ie KCID')

    while True:
        try:
            print('Enter the model run date and model run hour you want to use')
            yr = (input('Year: '))
            mo = (input('Month: '))
            dy = (input('Day: '))
            hr = (input('Model Hour: '))
            fcst = int(input('Enter the forcast hour to use'))
            if len(mo) <2:
                mo = '0'+mo
            if len(dy) <2:
                dy = '0'+dy
            if len(hr) <2:
                hr = '0'+hr
            print(yr,mo,dy,hr)
            bufkit_data = spy.get_bufkit_data(mdl, loc, fcst, yr, mo, dy, hr)
            spy.build_sounding(bufkit_data, dark_mode=True, special_parcels='simple', map_zoom=0)
        except ValueError:
            print('Please enter a valid data')
    


In [ ]:
if datkind == 1:
    print('Getting ROAB Soundings!')
    ROAB()
elif datkind == 2:
    print('Getting ACARS Sounding data!')
    ACARS()
elif datkind == 3:
    print('Getting Model Data!')
    Model()

In [ ]:
h = [["sb_ia_ecape"], ["sb_ps_ecape", "sb_ps_cape"]]
bufkit_data = spy.get_bufkit_data('rap', 'kcid', 3, '2025', '05', '15', '15')

while True:
    try:
        modinp = (input('Modify the surface conditions? <Y or N>')).capitalize()
        if modinp == "Y" or modinp == "N":
            break
        else:
            print('Please only use "Y/y" or "N/n" ')
    except ValueError:
        print("Invalid input. Please enter an integer.")

if modinp == "Y":
    T_F = float(input('enter the surface Temperature (ºF)'))
    Td_F = float(input('enter the surface Dew Point (ºF)'))
    T = round((T_F * units.degF).to(units.degC),1).magnitude
    Td = round((Td_F * units.degF).to(units.degC),1).magnitude
    #print(round(T,1).magnitude,round(Td,1).magnitude)
    print(T,Td)
    spy.build_sounding(bufkit_data, dark_mode=True, special_parcels='simple', map_zoom=0, modify_sfc={'T':T, 'Td':Td})
    #spy.build_sounding(bufkit_data, dark_mode=True, special_parcels=[["sb_ia_ecape"],["sb_ps_ecape", "sb_ps_cape"]], radar_time='sounding', map_zoom=0, modify_sfc={'T':T, 'Td':Td})
else:
    spy.build_sounding(bufkit_data, dark_mode=True, special_parcels='simple', map_zoom=0)

In [ ]:
#spy.get_bufkit_data('model', 'station', forecast_hr, 'run_year', 'run_month', 'run_day', 'run_hour')
#bufkit_data = spy.get_bufkit_data('hrrr', 'OUN', 0, '2024', '05', '07', '00')
#spy.build_sounding(clean_data, color_blind=False, dark_mode=False, storm_motion='right_moving', special_parcels=None, show_radar=True, radar_time='sounding', map_zoom=2, modify_sfc=None, save=False, filename='sounderpy_sounding')

In [ ]:
#spy.get_model_data(RAP, latlon, year, month, day, hour, dataset=None, box_avg_size=0.10, hush=False, clean_it=True)
h = [["sb_ia_ecape"], ["sb_ps_ecape", "sb_ps_cape"]]
bufkit_data = spy.get_bufkit_data('rap', 'kcid', 3, '2025', '05', '15', '15')
#spy.build_sounding(bufkit_data, dark_mode=True, special_parcels=special_parcels, radar_time='sounding', map_zoom=2, modify_sfc=None,modify_sfc={'T':21, 'Td':19, 'ws': 30, 'wd':270})
#spy.build_sounding(bufkit_data, dark_mode=True, special_parcels=[["sb_ia_ecape"],["sb_ps_ecape", "sb_ps_cape"]], radar_time='sounding', map_zoom=2, modify_sfc={'T':28.9, 'Td':20})
spy.build_sounding(bufkit_data, dark_mode=True, special_parcels='simple')
#spy.build_sounding(bufkit_data)

In [ ]:
#Composite Soundings

data_list = []
'''
for hour in ['00', '01', '02', '03', '04', '05', '06']:
    cd = spy.get_bufkit_data('hrrr', 'kcid', 5, '2025', '05', '15', hour)#, hush=True)
    data_list.append(cd)
'''
for hour in [0, 1, 2, 3, 4, 5, 6]:
    cd = spy.get_bufkit_data('hrrr', 'kcid', hour, '2025', '05', '15', '12')#, hush=True)
    data_list.append(cd)

# and make it dark-mode for fun!
spy.build_composite(data_list, dark_mode=True, lw_to_use=[4 for cd in data_list])

In [ ]:


spy.build_vad_hodograph(vad_data, dark_mode=False, storm_motion='right_moving', sr_hodo=False, save=False, filename='sounderpy_sounding')

In [ ]:

acars_conn = spy.acars_data('2025', '05', '19', '15')

# List profiles
t = acars_conn.list_profiles()

print(t)

In [ ]:
# Start by setting up an 'ACARS connection'
hrstart = 15
#for i in range(4):
#    print(hrstart-i)
dlst=[]
hours = [hrstart-i for i in range(4)]
for i in hours:
    print(i)
    d = spy.acars_data('2025', '05', '15', str(i))
    #dat = d.list_profiles()
    dlst.append(d.list_profiles())

dat = [i for x in dlst for i in x]
# List profiles
#acars_conn.list_profiles()
print(dat)
res = any('DSM' in i for i in dat)
print(res)

#if any(i=='DSM' for i in dat):
#    print(dat.index('DSM'))


if any(i =='DSM' for i in dat):
    print(dat.index(i))

In [ ]:
acars = acars_conn.get_profile('DSM_1710')
spy.build_sounding(acars, dark_mode=True, special_parcels='simple')

In [ ]:
#dat = spy.get_obs_data('DVN', '2025', '05', '19', 12, hush=True, clean_it=True) [44.92, -84.72]
#      Models -----   spy.get_bufkit_data('model', 'station', forecast_hr, 'run_year', 'run_month', 'run_day', 'run_hour')
#      Models -----   spy.get_model_data('model', latlon, year, month, day, hour, dataset=None, box_avg_size=0.10, hush=False, clean_it=True)

dat = spy.get_bufkit_data('rap', 'KGGI', 10, '2025', '05', '19', '12')

spy.build_sounding(dat, dark_mode=True, special_parcels='simple', map_zoom=0)
